In [ ]:
## find gene overlap between monkey data and mouse references

## mouse data vs mouse references

In [1]:
import sys
import scanpy as sc
import anndata
import pandas as pd
import numpy as np
import os

data_type = 'float32'



import matplotlib as mpl
from matplotlib import rcParams
import matplotlib.pyplot as plt
import seaborn as sns

# silence scanpy that prints a lot of warnings
import warnings
warnings.filterwarnings('ignore')
rcParams['pdf.fonttype'] = 42 # enables correct plotting of text for PDFs

In [2]:
results_folder = '/home/ssobti/projects/farnaz_spatial/output_data/monkey/cell2location/multi_ref_annot'

# create paths and names to results folders for reference regression and cell2location models
ref_run_name = f'{results_folder}/reference_signatures'
#run_name = f'{results_folder}/cell2location_map'

In [3]:
## borrow the combined reference from the mouse data
combined_refs = anndata.read_h5ad('/home/ssobti/projects/farnaz_spatial/data/mouse_data/combined_and_independent_refs/combined_refs_raw_data_seurat.h5ad')

In [4]:
## load in mouse data
# Set paths to data and results used through the document:
sp_data_folder = '/home/ssobti/projects/farnaz_spatial/data/mouse_data/'

In [5]:
def read_and_qc(sample_name, path=sp_data_folder):
    r""" This function reads the data for one 10X spatial experiment into the anndata object.
    It also calculates QC metrics. Modify this function if required by your workflow.

    :param sample_name: Name of the sample
    :param path: path to data
    """

    adata = sc.read_visium(path + str(sample_name) + '/outs',
                           count_file='filtered_feature_bc_matrix.h5', load_images=True)
    adata.obs['sample'] = sample_name
    adata.var['SYMBOL'] = adata.var_names
    adata.var.rename(columns={'gene_ids': 'ENSEMBL'}, inplace=True)
    adata.var_names = adata.var['ENSEMBL']
    adata.var.drop(columns='ENSEMBL', inplace=True)

    # Calculate QC metrics
    from scipy.sparse import csr_matrix
    adata.X = adata.X.toarray()
    sc.pp.calculate_qc_metrics(adata, inplace=True)
    adata.X = csr_matrix(adata.X)
    adata.var['mt'] = [gene.startswith('mt-') for gene in adata.var['SYMBOL']]
    adata.obs['mt_frac'] = adata[:, adata.var['mt'].tolist()].X.sum(1).A.squeeze()/adata.obs['total_counts']

    # add sample name to obs names
    adata.obs["sample"] = [str(i) for i in adata.obs['sample']]
    adata.obs_names = adata.obs["sample"] \
                          + '_' + adata.obs_names
    adata.obs.index.name = 'spot_id'

    return adata

def select_slide(adata, s, s_col='sample'):
    r""" This function selects the data for one slide from the spatial anndata object.

    :param adata: Anndata object with multiple spatial experiments
    :param s: name of selected experiment
    :param s_col: column in adata.obs listing experiment name for each location
    """

    slide = adata[adata.obs[s_col].isin([s]), :]
    s_keys = list(slide.uns['spatial'].keys())
    s_spatial = np.array(s_keys)[[s in k for k in s_keys]][0]

    slide.uns['spatial'] = {s_spatial: slide.uns['spatial'][s_spatial]}

    return slide

#######################
# Read the list of spatial experiments
sample_data = pd.read_csv(sp_data_folder + 'visium.csv')

# Read the data into anndata objects
slides = []
for i in sample_data['sample_name']:
    slides.append(read_and_qc(i, path=sp_data_folder))

# Combine anndata objects together
adata_vis = slides[0].concatenate(
    slides[1:],
    batch_key="sample",
    uns_merge="unique",
    batch_categories=sample_data['sample_name'],
    index_unique=None
)
#######################

In [6]:
len(set(combined_refs.var['features']).intersection(set(adata_vis.var['SYMBOL'])))

14097

## monkey data vs mouse references

In [7]:
# Set paths to data and results used through the document:
sp_data_folder = '/home/ssobti/projects/farnaz_spatial/data/monkey_data/unpacked_data/'

In [8]:
def read_and_qc(sample_name, path=sp_data_folder):
    r""" This function reads the data for one 10X spatial experiment into the anndata object.
    It also calculates QC metrics. Modify this function if required by your workflow.

    :param sample_name: Name of the sample
    :param path: path to data
    """

    adata = sc.read_visium(path + str(sample_name) + '/outs',
                           count_file='filtered_feature_bc_matrix.h5', load_images=True)
    adata.obs['sample'] = sample_name
    adata.var['SYMBOL'] = adata.var_names
    adata.var.rename(columns={'gene_ids': 'ENSEMBL'}, inplace=True)
    adata.var_names = adata.var['ENSEMBL']
    adata.var.drop(columns='ENSEMBL', inplace=True)

    # Calculate QC metrics
    from scipy.sparse import csr_matrix
    adata.X = adata.X.toarray()
    sc.pp.calculate_qc_metrics(adata, inplace=True)
    adata.X = csr_matrix(adata.X)
    adata.var['mt'] = [gene.startswith('mt-') for gene in adata.var['SYMBOL']]
    adata.obs['mt_frac'] = adata[:, adata.var['mt'].tolist()].X.sum(1).A.squeeze()/adata.obs['total_counts']

    # add sample name to obs names
    adata.obs["sample"] = [str(i) for i in adata.obs['sample']]
    adata.obs_names = adata.obs["sample"] \
                          + '_' + adata.obs_names
    adata.obs.index.name = 'spot_id'

    return adata

def select_slide(adata, s, s_col='sample'):
    r""" This function selects the data for one slide from the spatial anndata object.

    :param adata: Anndata object with multiple spatial experiments
    :param s: name of selected experiment
    :param s_col: column in adata.obs listing experiment name for each location
    """

    slide = adata[adata.obs[s_col].isin([s]), :]
    s_keys = list(slide.uns['spatial'].keys())
    s_spatial = np.array(s_keys)[[s in k for k in s_keys]][0]

    slide.uns['spatial'] = {s_spatial: slide.uns['spatial'][s_spatial]}

    return slide

#######################
# Read the list of spatial experiments
sample_data = pd.read_csv(sp_data_folder + 'visium.csv')

# Read the data into anndata objects
slides = []
for i in sample_data['sample_name']:
    slides.append(read_and_qc(i, path=sp_data_folder))

#### change the slide identity key in .uns so they aren't the repeats between slides ####
## new names:
## 096-count-A1: 1149_G
## 096-count-D1: 1572_G
## 327-count-A1: 1080_T
## 327-count-D1: 1080_B

slides[0].uns['spatial']['1149G_A1_096'] = slides[0].uns['spatial'].pop('count-A1')
slides[1].uns['spatial']['1572G_D1_096'] = slides[1].uns['spatial'].pop('count-D1')
slides[2].uns['spatial']['1080T_A1_327'] = slides[2].uns['spatial'].pop('count-A1')
slides[3].uns['spatial']['1080B_D1_327'] = slides[3].uns['spatial'].pop('count-D1')

# Combine anndata objects together
adata_vis = slides[0].concatenate(
    slides[1:],
    batch_key="sample",
    uns_merge="unique",
    batch_categories=sample_data['sample_name'],
    index_unique=None
)
#######################

In [9]:
len(set(combined_refs.var['features']).intersection(set(adata_vis.var['SYMBOL'])))

11

In [10]:
adata_vis.var

,feature_types,genome,SYMBOL,mt,n_cells_by_counts-1080B_D1_327,mean_counts-1080B_D1_327,log1p_mean_counts-1080B_D1_327,pct_dropout_by_counts-1080B_D1_327,total_counts-1080B_D1_327,log1p_total_counts-1080B_D1_327,...,log1p_mean_counts-1149G_A1_096,pct_dropout_by_counts-1149G_A1_096,total_counts-1149G_A1_096,log1p_total_counts-1149G_A1_096,n_cells_by_counts-1572G_D1_096,mean_counts-1572G_D1_096,log1p_mean_counts-1572G_D1_096,pct_dropout_by_counts-1572G_D1_096,total_counts-1572G_D1_096,log1p_total_counts-1572G_D1_096
ENSEMBL,,,,,,,,,,,,,,,,,,,,,
ENSG00000187634,Gene Expression,GRCh38,SAMD11,False,105,0.041590,0.040748,96.135444,113.0,4.736198,...,0.003692,99.768840,8.0,2.197225,119,0.038473,0.037751,96.423204,128.0,4.859812
ENSG00000188976,Gene Expression,GRCh38,NOC2L,False,82,0.031285,0.030805,96.981965,85.0,4.454347,...,0.000000,100.000000,0.0,0.000000,59,0.018635,0.018464,98.226631,62.0,4.143135
ENSG00000187961,Gene Expression,GRCh38,KLHL17,False,104,0.040118,0.039334,96.172249,109.0,4.700480,...,0.006911,99.352751,15.0,2.772589,177,0.055906,0.054399,94.679892,186.0,5.231109
ENSG00000187583,Gene Expression,GRCh38,PLEKHN1,False,6,0.002208,0.002206,99.779168,6.0,1.945910,...,0.000924,99.953768,2.0,1.098612,33,0.009919,0.009870,99.008115,33.0,3.526361
ENSG00000187642,Gene Expression,GRCh38,PERM1,False,17,0.006257,0.006237,99.374310,17.0,2.890372,...,0.000000,100.000000,0.0,0.000000,8,0.002405,0.002402,99.759543,8.0,2.197225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000212907,Gene Expression,GRCh38,MT-ND4L,False,0,0.000000,0.000000,100.000000,0.0,0.000000,...,0.001386,99.861304,3.0,1.386294,1,0.000301,0.000301,99.969943,1.0,0.693147
ENSG00000198886,Gene Expression,GRCh38,MT-ND4,False,0,0.000000,0.000000,100.000000,0.0,0.000000,...,0.000462,99.953768,1.0,0.693147,0,0.000000,0.000000,100.000000,0.0,0.000000
ENSG00000198786,Gene Expression,GRCh38,MT-ND5,False,0,0.000000,0.000000,100.000000,0.0,0.000000,...,0.000000,100.000000,0.0,0.000000,0,0.000000,0.000000,100.000000,0.0,0.000000


In [11]:
import gseapy as gp
from gseapy import Biomart
bm = Biomart()
# note the dataset and attribute names are different
m2h = bm.query(dataset='mmusculus_gene_ensembl',
               attributes=['ensembl_gene_id','external_gene_name',
                           'hsapiens_homolog_ensembl_gene',
                           'hsapiens_homolog_associated_gene_name'])

In [13]:
# get a dict symbol mappings
m2h_dict = {}
for i, row in m2h.loc[:,["external_gene_name", "hsapiens_homolog_associated_gene_name"]].iterrows():
    if row.isna().any(): continue
    m2h_dict[row['external_gene_name']] = row["hsapiens_homolog_associated_gene_name"]
# read gmt file into dict
#kegg = gp.read_gmt(path="tests/extdata/enrichr.KEGG_2016.gmt")
#print(kegg['MAPK signaling pathway Homo sapiens hsa04010'][:10])

In [23]:
m2h_genes_input = combined_refs.var['features']
m2h_genes_output = []

for gene in m2h_genes_input:
    if gene in m2h_dict.keys():
        m2h_genes_output.append(m2h_dict[gene])
    else:
        m2h_genes_output.append('NA')


In [30]:
## percent of mouse genes that didn't map to human gene
100*sum(np.array(m2h_genes_output) == 'NA')/len(m2h_genes_output)

16.40173410404624